In [2]:
import langsmith
from langchain import chat_models, prompts, smith
from langchain.schema import output_parser

from langsmith.evaluation import evaluate
from langsmith.schemas import Run, Example
import openai
from langsmith.wrappers import wrap_openai

from thesis2024.utils import init_llm_langsmith

import uuid


langsmith_name = "Langsmith Eval Experiment 1"
_ = init_llm_langsmith(llm_key=3, temp=0.5, langsmith_name=langsmith_name)


uid = uuid.uuid4()
client = langsmith.Client()

In [ ]:
"""Prompt for the Teaching Agent System."""
def build_tas_prompt(self):
    """Build the agent prompt."""
    system_message = """You will interact with a student who has no prior knowledge of the subject."""
    course = """Mathematics 1."""
    subject = """All subjects."""

    prompt_hub_template = hub.pull("augustsemrau/react-teaching-chat").template
    prompt_template = PromptTemplate.from_template(template=prompt_hub_template)
    prompt = prompt_template.partial(system_message=system_message, course_name=course, subject_name=subject)
    return prompt

In [5]:
dataset_name = "TAS v0 Conversations"
openai_client = wrap_openai(openai.Client())

def output_dataset(inputs: dict) -> dict:
    """Extract entire chat history from dataset."""
    chat_hist = inputs["chat_history"]
    chat_string = ""
    for message in chat_hist:
        if message['type'] == "ai":
            chat_string += f"Teaching Assistant: {message['content']}\n\n"
        else:
            chat_string += f"Student: {message['content']}\n\n"
    return {"chat_history": chat_string}

def is_answered(run: Run, example: Example) -> dict:

    # Get outputs
    conversation = run.outputs.get("chat_history")

    # Check if the student_answer is an empty string
    if not conversation:
        return {"key": "is_answered" , "score": 0}
    else:
        return {"key": "is_answered" , "score": 1}

def conversation_length(run: Run, example: Example) -> dict:

    # Get outputs
    conversation = run.outputs.get("chat_history")
    print(conversation)
    return {"key": "conversation_lengthd" , "score": len(conversation)}


# Evaluators
qa_evalulator = [is_answered, conversation_length]

# Run
experiment_results = evaluate(
    output_dataset,
    data=dataset_name,
    evaluators=qa_evalulator,
    experiment_prefix="test-dbrx-qa-custom-eval-is-answered",
    # Any experiment metadata can be specified here
    metadata={"variant": "stuff website context into gpt-3.5-turbo",},
)


View the evaluation results for experiment: 'test-dbrx-qa-custom-eval-is-answered-0f7f01bb' at:
https://smith.langchain.com/o/f6c953c1-69e2-5115-bc32-1c4edca6df0d/datasets/5672906f-7370-47d4-a6be-b959bb88caf2/compare?selectedSessions=aad5c241-40a0-4033-9b16-ce1c6b2b50fe




0it [00:00, ?it/s]

[{'type': 'human', 'content': 'Hello, I am August!', 'example': False, 'additional_kwargs': {}, 'response_metadata': {}}, {'type': 'ai', 'content': 'Hello August! How can I assist you today with Mathematics 1?', 'example': False, 'tool_calls': [], 'additional_kwargs': {}, 'response_metadata': {}, 'invalid_tool_calls': []}, {'type': 'human', 'content': 'Can you explain me how ADAM optimization works?', 'example': False, 'additional_kwargs': {}, 'response_metadata': {}}, {'type': 'ai', 'content': 'I can explain how ADAM optimization works. ADAM (Adaptive Moment Estimation) is an optimization algorithm that is used to update the parameters of a neural network based on the gradients of the loss function. It combines the advantages of two other optimization algorithms, RMSprop and AdaGrad. ADAM uses adaptive learning rates for each parameter, which helps converge faster and handle sparse gradients. The algorithm computes the first and second moments of the gradients and uses them to update 

In [16]:



# Define your runnable or chain below.
prompt = prompts.ChatPromptTemplate.from_messages(
  [
    ("system", "You should extract the conversation history from the input."),
    ("human", "{input}")
  ]
)
llm = chat_models.ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
chain = prompt | llm | output_parser.StrOutputParser()

# Define the evaluators to apply
eval_config = smith.RunEvalConfig(
    evaluators=[
        # "cot_qa",
        # smith.RunEvalConfig.LabeledCriteria("conciseness"),
        # smith.RunEvalConfig.LabeledCriteria("relevance"),
        # smith.RunEvalConfig.LabeledCriteria("coherence"),
        smith.RunEvalConfig.LabeledCriteria("harmfulness"),
        # smith.RunEvalConfig.LabeledCriteria("maliciousness"),
        # smith.RunEvalConfig.LabeledCriteria("helpfulness"),
        # smith.RunEvalConfig.LabeledCriteria("controversiality"),
        # smith.RunEvalConfig.LabeledCriteria("misogyny"),
        # smith.RunEvalConfig.LabeledCriteria("criminality"),
        # smith.RunEvalConfig.LabeledCriteria("insensitivity")
    ],
    # custom_evaluators=[smith.RunEvalConfig.LabeledCriteria("Conversation Evaluator")],
    eval_llm=chat_models.ChatOpenAI(model="gpt-4-turbo-2024-04-09", temperature=0),
    input_key="input",
    prediction_key="output",
    # reference_key="chat_history",
)


client = langsmith.Client()
chain_results = client.run_on_dataset(
    dataset_name="TAS v1 Conversations",
    llm_or_chain_factory=chain,
    evaluation=eval_config,
    project_name="test-internal-example-47",
    concurrency_level=5,
    verbose=True,
)

View the evaluation results for project 'test-internal-example-47' at:
https://smith.langchain.com/o/f6c953c1-69e2-5115-bc32-1c4edca6df0d/datasets/391da789-2074-43b9-a0eb-2970436e6929/compare?selectedSessions=cf78100b-890d-4ade-b44e-5d29c8c29844

View all tests for Dataset TAS v1 Conversations at:
https://smith.langchain.com/o/f6c953c1-69e2-5115-bc32-1c4edca6df0d/datasets/391da789-2074-43b9-a0eb-2970436e6929
[------------------------------------------------->] 3/3

,feedback.harmfulness,error,execution_time,run_id
count,3.0,0,3.000000,3
unique,NaN,0,NaN,3
top,NaN,NaN,NaN,c9ad869d-e574-4622-be07-d6b8b4d10e43
freq,NaN,NaN,NaN,1
mean,0.0,NaN,2.013774,NaN
std,0.0,NaN,0.822373,NaN
min,0.0,NaN,1.155348,NaN
25%,0.0,NaN,1.623366,NaN
50%,0.0,NaN,2.091383,NaN
75%,0.0,NaN,2.442988,NaN
